In [1]:
import pandas as pd
import numpy as np
import re
import os
from datetime import datetime
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000

In [2]:
def transform_date(df):
    """
    Приводим дату к общему формату
    Все к datetime
    """
    timestamp_df = df[df['Дата термообработки'].apply(type)==pd._libs.tslib.Timestamp]
    last_df = df[df['Дата термообработки'].apply(type)!=pd._libs.tslib.Timestamp]
    last_df['Дата термообработки'] = last_df['Дата термообработки'].apply(
        lambda x:datetime.strptime(x, "%d.%m.%Y"))
    timestamp_df['Дата термообработки'] = timestamp_df['Дата термообработки'].apply(
        pd._libs.tslib.Timestamp.to_pydatetime)
    last_df['Дата термообработки'] = last_df['Дата термообработки'].apply(change_to17)
    print(last_df[last_df['Дата термообработки'].apply(lambda x:x.year)>2018].shape)
    last_df.shape
    df = pd.concat([last_df, timestamp_df])
    df.reset_index(drop=True, inplace=True)
    return df

In [3]:
def get_dicts_of_mode_files(dir_name):
    """Возвращает словари файлов с режимами для обучения и для валидации

    Keyword arguments:
    dir_name -- путь к папке в которой лежат файлы с режимами
    
    Возвращает: два словаря датафреймов
    """
    file_names = os.listdir(dir_name)
    # файлы для валидации в словарь (пока что два файла)
    valid_files = dict()
    valid_files[file_names[0]] = pd.read_excel(dir_name+'\\'+file_names[0],skiprows=1)
    valid_files['сводка февраль 2018.xls'] = pd.read_excel(dir_name+'\\'+'сводка февраль 2018.xls',skiprows=1)
#     print('validation:', valid_files.keys())
    file_names.remove('сводка февраль 2018.xls')
    # файлы для обучения в словарь 
#     print('train:')
    mode_files = dict()
    for name in file_names[1:]:
        mode_files[name] = pd.read_excel(dir_name+'\\'+name,skiprows=1)
#         print(name)
    # Удаление лишнего столбца с комментариями из файла
    del mode_files['сводная 2014_v2.xls']['Unnamed: 42']
    return [mode_files, valid_files]

In [4]:
def rep_part(ls):
    """Удаляет .0 в номере партии и возвращает список без него"""
    for x in ls:
        x = str(x)
        x = x.replace('.0', '')
    return ls

In [5]:
def gen_batch_number_list(files_dict):
    """Создает список номеров партий из правой части таблицы

    Keyword arguments:
    files_dict -- словарь датафреймов, из которых брать номера партий (ключ в словаре - нназвание файла)
    
    Возвращает: list номеров партий
    """
    batch_number_list = []
    for mode in files_dict.values():
        for x in mode['№ партии.1']:
            batch_number_list.append(str(x).replace('.0', ''))

    batch_number_list = rep_part(batch_number_list)
    print(len(batch_number_list))
    return batch_number_list

In [6]:
def clean_dict(files_dict):
    """Убирает заголовки у датафреймов из словаря, удаляет лишние столбцы

    Keyword arguments:
    files_dict -- словарь файлов (ключ в словаре - нназвание файла)
    
    Возвращает: list номеров партий
    """
    # убираем заголовки
    tmp = dict()
    for name, mode in zip(files_dict.keys(), files_dict.values()):
        tmp[name] = mode.copy()
        tmp[name].columns = range(mode.shape[1])       
   
    return tmp

In [7]:
# def clean_dict(files_dict):
#     """
#     Убирает заголовки у датафреймов из словаря, удаляет лишние столбцы

#     Keyword arguments:
#     files_dict -- словарь файлов (ключ в словаре - нназвание файла)
    
#     Возвращает: list номеров партий
#     """   
#     # убираем последние пять столбцов из всех таблиц
#     for mode in files_dict.values():
#         mode.drop(mode.columns[-5:], axis=1, inplace=True)
#     return tmp

In [8]:
# Чистим Скорость прохождения трубы через спрейер, м/с
def clean_u_sp(x):
    x = str(x)
    x = x.replace(u'сек', '')
    x = x.replace(u'ек', '')
    x = x.replace(u'\мс', '')
    x = x.replace(u'm/c', '')
    x = x.replace(u'М/С', '')
    x = x.replace(u'М\С', '')
    x = x.replace(u'nan', '')
    x = x.replace(u'м/с', '')
    x = x.replace(u'м\с', '')
    x = x.replace(u',', u'.')
    x = x.replace(u'м\\', '')
    x = x.replace(u'м/', '')
    x = x.replace(u'*****', u'')
    x = x.replace(u'****', u'')
    x = x.replace(u'***', u'')
    x = x.replace(u'29.03.2017', u'')
    x = x.replace(',','.')
    x = x.replace('041','0.41')
    x = x.replace('НЕТ','')
    x = x.replace('нет','')  
    x = x.replace('м','')
    x = x.replace('021', '0.21')
    x = x.replace('1.8', '0.18')
    x = x.replace('2.0', '0.2')
    x = x.replace('012', '0.12')
    if x == '45':
        return '0.45'
    if x == '064':
        return '0.64'
    x = x.replace(u'0.7.', '0.7')
    try:
        y = x.split('-')
        x = (float(y[0])+float(y[1]))/2
    except:
        pass
    return x

In [9]:
def return_headers(df, headers_list, batch_list):
    """Возвращаем заголовки и номера партий, удаляем партии без номеров"""
    df.columns = headers_list[:len(df.columns)]
    df['№ партии'] = batch_list
    df = df[df['№ партии'] != 'nan']
    print(df.shape)
    return df

In [10]:
def get_df_modes(dict_modes, headers_file_name):
    # Сохраняем столбец с номером партии в переменную batch_number_list 
    batch_number_list = gen_batch_number_list(dict_modes)
    
     # убираем последние пять столбцов из всех таблиц
    for mode in dict_modes.values():
        mode.drop(list(mode.columns[-5:-2])+[mode.columns[-1]], axis=1, inplace=True)
    
    # Сохраняем заголовки
    header = dict_modes[headers_file_name].columns
    # удаляем заголовки и лишние столбцы
    dict_modes_tmp_copy = clean_dict(dict_modes)
    df_modes = pd.DataFrame()
    # слияние всех режимов
    for df in dict_modes_tmp_copy.values():
        df_modes = pd.concat([df_modes,df])
    # Сбрасываем индекс, чтобы не было повторяющихся
    df_modes.reset_index(drop=True, inplace=True)
#     print(df_modes.shape)
    # Возвращаем заголовки и номера партий, удаляем партии без номеров
    df_modes = return_headers(df_modes, header, batch_number_list)
#     print(df_modes.shape)
    return df_modes

In [11]:
# def get_df_modes(dict_modes, headers_file_name):
#     # Сохраняем столбец с номером партии в переменную batch_number_list 
#     batch_number_list = gen_batch_number_list(dict_modes)
       
#     dict_modes_tmp_copy = clean_dict(dict_modes)
    
#     # Сохраняем заголовки
#     header = dict_modes[headers_file_name].columns
#     # удаляем заголовки и лишние столбцы
#     # убираем заголовки
#     tmp = dict()
#     for name, mode in zip(files_dict.keys(), files_dict.values()):
#         tmp[name] = mode.copy()
#         tmp[name].columns = range(mode.shape[1])    

#     df_modes = pd.DataFrame()
#     # слияние всех режимов
#     for df in dict_modes_tmp_copy.values():
#         df_modes = pd.concat([df_modes,df])
#     # Сбрасываем индекс, чтобы не было повторяющихся
#     df_modes.reset_index(drop=True, inplace=True)
# #     print(df_modes.shape)
#     # Возвращаем заголовки и номера партий, удаляем партии без номеров
#     df_modes = return_headers(df_modes, header, batch_number_list)
# #     print(df_modes.shape)
#     return df_modes

In [12]:
# Разделяем диаметр и толщину стенки и чистим их
def split_d(x):
    x = str(x)
    x = x.replace(',', '.')
    return re.split('х|Х|x|X', x)[0]

def split_s(x):
    x = str(x)
    x = x.replace(',', '.')
    try:
        return re.split('х|Х|x|X', x)[1]
    except:
        return x
    
def clean_d_s(x):
    x = str(x)
    x = x.replace('п', '')
    x = x.replace('П', '')
    x = x.replace('nan', '')
    x = x.replace(',', '.')
    return x



In [13]:
# Объединение марок стали
def rep_steel(x):
    x = str(x)
    x = x.replace('092ГС','09Г2С')
    x = x.replace('13 ХФА','13ХФА')
    x = x.replace('13хфа','13ХФА')
    x = x.replace('13ХФА ','13ХФА')
    x = x.replace('13хфа-3','13ХФА-3')
    x = x.replace('15Х13H2MБ','15Х13H2МБ')
    x = x.replace('15х5мфбч','15Х5МФБЧ')
    x = x.replace('18х3мфб','18Х3МФБ')
    x = x.replace('18Х3МФБ ','18Х3МФБ')
    x = x.replace('18х3ХМФБ','18Х3МФБ')
    x = x.replace('18хзмфб','18Х3МФБ')
    x = x.replace('18хмфб','18ХМФБ')
    x = x.replace('18ХМФБ ','18ХМФБ')
    x = x.replace('20А простая','20А-пр')
    x = x.replace('20А-ПР','20А-пр')
    x = x.replace('20 А','20А')    
    x = x.replace('20A','20А') 
    x = x.replace('20А ','20А')
    x = x.replace('20 АУ','20А-У')
    x = x.replace('20АУ','20А-У')
    x = x.replace('30Г2 ','30Г2')
    x = x.replace('30Г2.','30Г2')
    x = x.replace('30г2-2','30Г2-2')
    x = x.replace('30г2ф','30Г2Ф')
    x = x.replace('30г2','30Г2')
    x = x.replace('30хгма','30ХГМА')
    x = x.replace('30ХГМА1','30ХГМА-1')
    x = x.replace('30хма','30ХМА')
    x = x.replace('32г1','32Г1')
    x = x.replace('32г2','32Г2')
    x = x.replace('32х1мф-1','32Х1МФ-1')
    x = x.replace('32ха','32ХА')
    x = x.replace('37г2ф','37Г2Ф')
    x = x.replace('37г2Ф','37Г2Ф')
    x = x.replace('37Г2ф','37Г2Ф')
    x = x.replace('37Г2Ф ','37Г2Ф')
    x = x.replace('38Г2С ','38Г2С')
    x = x.replace('38г2с-4','38Г2С-4')
    x = x.replace('38Г2-С-9','38Г2С-9')
    x = x.replace('30Г2 ', '30Г2')
    return x


In [14]:
# Разносим шаг на закалочной и отпускной печах в разные столбцы и очищаем их
def clean_steps(x):
    x = str(x)
    x = x.replace(u'X', '/')
    x = x.replace(u'x', '/')
    x = x.replace(u'Х', '/')
    x = x.replace(u'х', '/')
    x = x.replace(',', '.')
    x = x.replace('\\', '/')
    x = x.replace('/-', '/0')
    x = x.replace('-/', '0/')
    x = x.replace('***', '0')
    x = x.replace('*', '0')
    x = x.replace('-', '/')
    x = x.replace('nan', '')
    return x

def convert_step0(x):
    return clean_steps(x).split('/')[0]
   
def convert_step1(x):
    try:
        return clean_steps(x).split('/')[1]
    except:
        return x

# df_modes = df_modes[df_modes[u'шаг балок закалочная/отпускная печи, сек']!='***']


In [15]:
# Чистим время выдержки в закалочной ванне, сек.
def clean_time_bath(x):
    x = str(x)
    x = x.replace(u'сек', '')
    x = x.replace(u' ', '')
    x = x.replace(u'.', '')
    return x

In [16]:
# Удаляем лишнии столбцы из датафрейма
l2del_modes = [u'время начала т/о',u'D/S',u'шаг балок закалочная/отпускная печи, сек', u'плавка',
      '2 зона уставка закалка', '2 зона фактическая температура закалка', '2 зона фактическая температура отпуск', 
      '4 зона фактическая температура отпуск', 'кол-во, шт.'
]

l2del_valid =[u'время начала т/о',u'D/S',u'шаг балок закалочная/отпускная печи, сек', u'плавка', 'кол-во, шт.', 
    'масса\nтн']

def del_cols_df(df, l2del):
    df[u'№ плавки'] = df[u'плавка']
    df = df.drop(l2del, axis = 1)
    return df


In [17]:
ls_vtr = [
    '1 зона по ВТР закалка',
    '2 зона по ВТР закалка',
    '3 зона по ВТР закалка',
    '1 зона ВТР и уставка отпуск',
    '2 зона ВТР и уставка отпуск',
    '4 зона ВТР и уставка отпуск'
]
# Очистка всех столбцов с температурами по втр 
def clean_vtr(x):
    x = str(x)
    x = x.replace('*****','')
    x = x.replace('****','')
    x = x.replace('***','')
    x = x.replace('-','')
    x = x.replace('50/*','')
    x = x.replace('нет','')
    x = x.replace('НЕТ','')
    x = x.replace('н/д','')
    return x


def clean_all(df, ls, apply_f):
    """Применяет к столбцам из списка ls датафрейма df метод apply_f"""
    for col in ls:
        df[col] = df[col].apply(apply_f)
    return df

In [18]:
ls_nd = [
    '3 зона ВТР и уставка отпуск',
    '5 зона ВТР и уставка отпуск',
    '1 зона уставка закалка',
    '1 зона фактическая температура закалка',
    '3 зона уставка закалка',
    '3 зона фактическая температура закалка',
    'Температура трубы на выгрузке из печи закалки',
    'Расход воды на закалочный спрейер, м3',
    't˚ C воды в закалочной ванне',
    '1 зона фактическая температура отпуск',
    '3 зона фактическая температура отпуск',
    '5 зона фактическая температура отпуск',
    'Температура трубы на выгрузке из печи отпуска'
]
# Чистим столбцы от н/д
def rep_nd(x):
    x = str(x)
    x = x.replace('н/д','')
    x = x.replace('.0', '')
    x = x.replace('НЕТ', '')
    x = x.replace('***', '')
#     x = x.replace('охлаждение','')
    return x

In [19]:
# Чистим время выдержки в закалочной ванне, сек.
def rep_time(x):
    x = str(x)
    x = x.replace('nan','')
    x = x.replace('НЕТ','')
    x = x.replace('нет','')
    x = x.replace('НОРМ','')
    x = x.replace('****','')
    x = x.replace('***','')
    x = x.replace('40-50','45')
    x = x.replace('сек','')
    x = x.replace('с','')
    x = x.replace(',','.')
    x = x.replace('11cек', '')
    x = x.replace('HET', '')
    return x

In [20]:
def rep_tempr(x):
    x = str(x)
    x = x.replace('******','')
    x = x.replace('*****','')
    x = x.replace('****','')
    x = x.replace('***','')
    x = x.replace('НОРМ-ИЯ', '')
    x = x.replace('норм-ция', '')
    x = x.replace('\\','/')
    x = x.replace('-','/')
    x = x.replace('<', '')
    x = x.replace('>', '')
    x = x.replace('Доотпуск', 'доотпуск')
    x = x.replace('до', '')
    x = x.replace('Бизнес Тренд', '')
    x = x.replace('БизнесТренд', '')
    x = x.replace('ВОЗДУХ', '')
    x = x.replace('Воздух', '')
    x = x.replace('воэдух', '')
    x = x.replace('высадка', '')
    x = x.replace('закалка', '')
    x = x.replace('НЕТ', '')
    x = x.replace('нет', '')
    x = x.replace('НЕТ', '') 
    x = x.replace('нет', '')
    x = x.replace('НОРМ', '')
    x = x.replace('нормализ.', '')
    x = x.replace('нормализация', '')
    x = x.replace('воздух', '')
    x = x.replace('61702', '')
    x = x.replace('опытные.','')
    x = x.replace('на е','')
    x = x.replace('норм/ия','')
    x = x.replace('нормал/ция','')
    x = x.replace('АЛ/ЦИЯ','')
    x = x.replace(',','.')
    y = x.split('/')
    if len(y)==1:
        return x
    fl = True
    tmpy= []
    for i in y:
        if len(i) == 4:
            y.append(i[0:2])
            y.append(i[2:4])
            y.remove(i)
    for i in y:
        try:
            tmpy.append(float(i))
        except:
            tmpy.append(i)
            fl = False
    if(fl):
        y = tmpy   
        x = np.mean(y)
    if x=='0':
        x=''
    return x


def clean_and_tofloat(df, ls, apply_f_ls):
    
    for col, apply_f in zip(ls,apply_f_ls):
        df[col] = df[col].apply(apply_f)
    df.drop(df[df[col]=='отпуск'].index, inplace = True, axis = 0)
    df[col] = df[col].apply(
                lambda y: str(y) if (y!='nan')&(y!='')&(y!=None)&(y!=' ') else None).astype(float)
    return df

In [21]:
def rep_tempr_bath(x):
    x = str(x)
    x = x.replace('Бизнес Тренд', '')
    x = x.replace('бизнес тренд', '')
    x = x.replace('высадка.', '')
    x = x.replace('высадка', '')
    x = x.replace('ВЫСАДКА', '')
    x = x.replace('Высадка', '')
    x = x.replace('высадка', '') 
    x = x.replace('доотпуск.', 'отпуск')
    x = x.replace('доотпуск', 'отпуск')
    x = x.replace('Доотпуск', 'отпуск')
    x = x.replace('отпуск.', 'отпуск')
    x = x.replace('на отпуск', 'отпуск')
    x = x.replace('в малахи', '')
    x = x.replace('ДООТПУСК', 'отпуск')
    x = x.replace('НЕТ', '')
    x = x.replace('нет', '')
    x = x.replace('НЕТ', '') 
    x = x.replace('НК 8500', '')
    x = x.replace('НК 9000', '')
    x = x.replace('нормализ.', '')
    x = x.replace('нормализация', '')
    x = x.replace('нормал-ция', '')
    x = x.replace('НОРМ-ИЯ', '')
    x = x.replace('норм-ция.', '')
    x = x.replace('норм-ция', '')
    x = x.replace('нор-ция', '')
    x = x.replace('нор/ция', '')
    x = x.replace('опытные.', '')
    x = x.replace('ОПЫТНЫЕ', '')
    x = x.replace('повторная', '')
    x = x.replace('повтор', '')
    x = x.replace('ПОВТОР', '')
    x = x.replace('Самара', '')
    x = x.replace('СИТЦ', '')
    x = x.replace('Трэнд', '')
    x = x.replace('ЧПТЗ', '')
    x = x.replace('ЧТПЗ', '')
    x = x.replace('30 СЕК.', '30')
    x = x.replace('2/я зак/ка', '')
    x = x.replace('ТРЕНД', '')
    x = x.replace('ПЛ.Р. Ц42/№10','')
    x = x.replace('Челябинск','')
    x = x.replace('нормал/ция', '')
    x = x.replace('HET', '')
    if x=='0':
        x=''
    if x=='.':
        x=''
    return x

In [22]:
def get_modes_dataframe():
    
    mode_files, valid_files = get_dicts_of_mode_files('../modes')
    headers_for_42_col_files = 'сводка 1 и 2 квартал 2017_v2.xls'
    df_modes = get_df_modes(mode_files, headers_for_42_col_files)

    headers_for_28_col_files = 'Копия сводка январь 2018.xls'
    df_valid = get_df_modes(valid_files, headers_for_28_col_files)
    df_modes['диаметр'] = df_modes[u'D/S'].apply(split_d)
    df_modes['толщина стенки'] = df_modes[u'D/S'].apply(split_s)
    df_modes['толщина стенки'] = df_modes['толщина стенки'].apply(clean_d_s)
    df_modes['толщина стенки'] = df_modes['толщина стенки'].apply(lambda x: x.replace('107.95', ''))
    df_modes['диаметр'] = df_modes['диаметр'].apply(clean_d_s)

    df_valid['диаметр'] = df_valid[u'D/S'].apply(split_d)
    df_valid['толщина стенки'] = df_valid[u'D/S'].apply(split_s)
    df_valid['толщина стенки'] = df_valid['толщина стенки'].apply(clean_d_s)
    df_valid['толщина стенки'] = df_valid['толщина стенки'].apply(lambda x: x.replace('107.95', ''))
    df_valid['диаметр'] = df_valid['диаметр'].apply(clean_d_s)
    
    
    df_modes['марка стали'] = df_modes['марка стали'].apply(rep_steel)
    df_valid['марка стали'] = df_valid['марка стали'].apply(rep_steel)
    
    df_modes[u'шаг балок закалочная печь, сек'] = df_modes[u'шаг балок закалочная/отпускная печи, сек'].apply(convert_step0)
    df_modes[u'шаг балок отпускная печь, сек'] = df_modes[u'шаг балок закалочная/отпускная печи, сек'].apply(convert_step1)

    df_valid[u'шаг балок закалочная печь, сек'] = df_valid[u'шаг балок закалочная/отпускная печи, сек'].apply(convert_step0)
    df_valid[u'шаг балок отпускная печь, сек'] = df_valid[u'шаг балок закалочная/отпускная печи, сек'].apply(convert_step1)
    
    col = 'Скорость прохождения трубы через спрейер, м/с' 
    df_modes[col] = df_modes[col].apply(clean_u_sp)
    df_valid[col] = df_valid[col].apply(clean_u_sp)
    
    
    col = u'время выдержки в закалочной ванне, сек.'
    df_modes[col] = df_modes[col].apply(clean_time_bath)
    df_valid[col] = df_valid[col].apply(clean_time_bath)
    
    df_modes = del_cols_df(df_modes, l2del_modes)
    df_valid = del_cols_df(df_valid, l2del_valid)
    
    df_modes = clean_all(df_modes, ls_vtr, clean_vtr)
    df_valid = clean_all(df_valid, ls_vtr, clean_vtr)
    
    df_modes = clean_all(df_modes, ls_nd, rep_nd)
    df_valid = clean_all(df_valid, ls_nd[:-11], rep_nd)
    
    col ='время выдержки в закалочной ванне, сек.'
    df_modes[col] = df_modes[col].apply(rep_time)
    df_valid[col] = df_valid[col].apply(rep_time)
    
    col = 't˚ C трубы после спреера'
    df_modes = clean_and_tofloat(df_modes, [col], [rep_tempr])
    df_valid = clean_and_tofloat(df_valid, [col], [rep_tempr])

    col = 't˚ C трубы после ванны'
    df_modes = clean_and_tofloat(df_modes, [col, col], [rep_tempr, rep_tempr_bath])
    df_valid = clean_and_tofloat(df_valid, [col, col], [rep_tempr, rep_tempr_bath])
  
    df_modes.drop(df_modes[df_modes['шаг балок закалочная печь, сек'] == '0'].index, inplace=True, axis=0)
    df_modes.drop(df_modes[df_modes['шаг балок отпускная печь, сек'] == '0'].index, inplace=True, axis=0)
    df_valid.drop(df_valid[df_valid['шаг балок закалочная печь, сек'] == '0'].index, inplace=True, axis=0)
    df_valid.drop(df_valid[df_valid['шаг балок отпускная печь, сек'] == '0'].index, inplace=True, axis=0)
    
    df_modes['№ партии'] = df_modes['№ партии'].astype(str).str.lower()
    df_modes['№ плавки'] = df_modes['№ плавки'].astype(str).str.lower()
    df_valid['№ партии'] = df_valid['№ партии'].astype(str).str.lower()
    df_valid['№ плавки'] = df_valid['№ плавки'].astype(str).str.lower()
    
    return [df_modes, df_valid]

In [ ]:
if __name__ == '__main__':
    get_modes_dataframe()